# Experiment Notebook: {Descriptive Experiment Keywords}

# Table of Contents
* [Experiment Summary](#Experiment-Summary)
* [Experiment Assumptions](#Experiment-Assumptions)
* [Experiment Setup](#Experiment-Setup)
* [Analysis 1: ...](#Analysis-1:-...)

# Experiment Summary 

The purpose of this notebook is to...

# Experiment Assumptions

See [assumptions document](../../ASSUMPTIONS.md) for further details.

# Experiment Setup

We begin with several experiment-notebook-level preparatory setup operations:

* Import relevant dependencies
* Import relevant experiment templates
* Create copies of experiments
* Configure and customize experiments 

Analysis-specific setup operations are handled in their respective notebook sections.

In [1]:
# Import the setup module:
# * sets up the Python path
# * runs shared notebook configuration methods, such as loading IPython modules
import setup

import copy
import logging
import numpy as np
import pandas as pd
import plotly.express as px

import experiments.notebooks.visualizations as visualizations
from experiments.run import run
from experiments.utils import display_code

time: 230 ms (started: 2022-12-07 17:10:09 +08:00)


In [2]:
# Enable/disable logging
logger = logging.getLogger()
logger.disabled = False

time: 10.5 ms (started: 2022-12-07 17:10:09 +08:00)


In [15]:
# Import experiment templates
import experiments.templates.polygon_time_domain_analysis as time_domain_analysis
import experiments.templates.polygon_cumulative_yield_analysis as cumulative_yield_analysis

time: 736 ms (started: 2022-12-07 17:20:12 +08:00)


In [4]:
# Inspect experiment template
display_code(time_domain_analysis)

"""
# Time Domain Analysis

Executes a time-domain simulation over a long time period of 3 years,
over all Ethereum network upgrade stages.
"""

import copy

import model.constants as constants
from model.stochastic_processes import create_stochastic_process_realizations
from model.types import Stage
from experiments.default_experiment import experiment


# Make a copy of the default experiment to avoid mutation
experiment = copy.deepcopy(experiment)

DELTA_TIME = constants.epochs_per_day  # epochs per timestep
SIMULATION_TIME_MONTHS = 1  # number of months
TIMESTEPS = constants.epochs_per_month * SIMULATION_TIME_MONTHS // DELTA_TIME


# Override default experiment Simulation and System Parameters related to timing
experiment.simulations[0].timesteps = TIMESTEPS
experiment.simulations[0].model.params.update({"dt": [DELTA_TIME]})

time: 38.9 ms (started: 2022-12-07 17:10:09 +08:00)


In [5]:
# Create a simulation for each analysis
simulation_1a = copy.deepcopy(time_domain_analysis.experiment.simulations[0])
simulation_1b = copy.deepcopy(cumulative_yield_analysis.experiment.simulations[0])

time: 11.2 ms (started: 2022-12-07 17:10:09 +08:00)


In [6]:
# Experiment configuration
simulation_1a.model.initial_state.update({})

simulation_1a.model.params.update({})

time: 10.5 ms (started: 2022-12-07 17:10:09 +08:00)


# Analysis 1: ...

{Analysis Description}

In [7]:
# Analysis-specific setup
normal_adoption = simulation_1a.model.params['validator_process'][0](_run=None, _timestep=None)
simulation_1a.model.params.update({
    'validator_process': [
        lambda _run, _timestep: normal_adoption,  # Normal adoption: current 6-month average active validators per epoch from The Graph Subgraph
        lambda _run, _timestep: normal_adoption * 0.5,  # Low adoption: 50%-lower scenario
        lambda _run, _timestep: normal_adoption * 1.5,  # High adoption: 50%-higher scenario
    ],  # New validators per epoch
})

time: 11.3 ms (started: 2022-12-07 17:10:09 +08:00)


In [8]:
# Experiment execution
df_1a, _exceptions = run(simulation_1a)

2022-12-07 17:10:09,903 - root - INFO - Running experiment
2022-12-07 17:10:09,906 - root - INFO - Starting simulation 0 / run 0 / subset 0
2022-12-07 17:10:09,909 - root - INFO - Starting simulation 0 / run 0 / subset 1
2022-12-07 17:10:09,912 - root - INFO - Starting simulation 0 / run 0 / subset 2
2022-12-07 17:10:09,914 - root - INFO - Experiment complete in 0.009525060653686523 seconds
2022-12-07 17:10:09,914 - root - INFO - Post-processing results
2022-12-07 17:10:09,967 - root - INFO - Post-processing complete in 0.052678823471069336 seconds
time: 76.9 ms (started: 2022-12-07 17:10:09 +08:00)


In [9]:
print(df_1a[["timestamp","subset","number_of_awake_validators","number_of_active_validators","average_effective_balance",'network_issuance','polygn_supply','polygn_staked','supply_inflation']])
for i in df_1a.columns:
    print(i)

                    timestamp  subset  number_of_awake_validators  \
1  2022-12-07 17:10:09.341894       0                         100   
2  2022-12-08 17:10:09.341894       0                         100   
3  2022-12-09 17:10:09.341894       0                         100   
4  2022-12-10 17:10:09.341894       0                         100   
5  2022-12-11 17:10:09.341894       0                         100   
..                        ...     ...                         ...   
88 2023-01-01 17:10:09.341894       2                         100   
89 2023-01-02 17:10:09.341894       2                         100   
90 2023-01-03 17:10:09.341894       2                         100   
91 2023-01-04 17:10:09.341894       2                         100   
92 2023-01-05 17:10:09.341894       2                         100   

    number_of_active_validators  average_effective_balance  network_issuance  \
1                           100               3.000000e+16     273788.568125   
2          

In [10]:
# Post-processing and visualizations
visualizations.plot_number_of_validators_per_subset(df_1a, scenario_names={0: "Normal Adoption", 1: "Low Adoption", 2: "High Adoption"})

time: 80.8 ms (started: 2022-12-07 17:10:10 +08:00)


In [11]:
visualizations.plot_yields_per_subset_subplots(
    df_1a,
    subplot_titles=['Normal Adoption', 'Low Adoption', 'High Adoption']
)

time: 89.9 ms (started: 2022-12-07 17:16:33 +08:00)


In [12]:
visualizations.plot_yields_per_subset(df_1a, scenario_names={0: "Normal Adoption", 1: "Low Adoption", 2: "High Adoption"})

time: 44.4 ms (started: 2022-12-07 17:17:11 +08:00)


In [13]:
visualizations.plot_cumulative_yields_per_subset(df_1a, simulation_1a.model.params["dt"][0], scenario_names={0: "Normal Adoption", 1: "Low Adoption", 2: "High Adoption"})

time: 32.1 ms (started: 2022-12-07 17:17:28 +08:00)


In [14]:
simulation_1b.model.params.update({
    'base_fee_process': [
        lambda _run, _timestep: 0,
        lambda _run, _timestep: 30,
    ],
    'priority_fee_process': [
        lambda _run, _timestep: 0,
        lambda _run, _timestep: 2,
    ],
})

NameError: name 'simulation_1b' is not defined

time: 123 ms (started: 2022-12-07 17:17:47 +08:00)
